In [ ]:
! pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

wassa_train_csv_path = '/content/drive/My Drive/WASSA_train_all.csv'
wassa_dev_csv_path = '/content/drive/My Drive/WASSA_dev_all.csv'
wassa_train = pd.read_csv(wassa_train_csv_path)
wassa_dev = pd.read_csv(wassa_dev_csv_path)

augmented_path = '/content/drive/My Drive/wassa_all_BertAug_5.csv'
aug_data = pd.read_csv(augmented_path,encoding='utf-8')
print(wassa_train.shape)
print(aug_data.shape)

(1860, 2)
(9300, 3)


In [7]:
parafrase = aug_data[['paraphrase', 'label']].copy()
parafrase = parafrase.drop_duplicates()
parafrase.rename(columns={"paraphrase": "essay","label":"emotion"}, inplace = True)

wassa_train = wassa_train.append(parafrase,ignore_index=True)
print(wassa_train.shape)

(11160, 2)


<ipython-input-7-787c2a156d3c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wassa_train = wassa_train.append(parafrase,ignore_index=True)


In [8]:
train_df = pd.DataFrame(wassa_train)
train_df.columns = ["text", "labels"]

eval_df = pd.DataFrame(wassa_dev)
eval_df.columns = ["text", "labels"]

In [9]:
import torch
from torch import cuda
import gc

def empty_cache(model, optimizer):
  device = 'cuda' if cuda.is_available() else 'cpu'
  with torch.no_grad():
      torch.cuda.empty_cache()
  gc.collect()

In [10]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, precision_score, recall_score

def f1_multiclass(labels, preds):
  return f1_score(labels, preds, average='macro')

def precision_multiclass(labels,preds):
  return precision_score(labels, preds, average='macro')

def recall_multiclass(labels,preds):
  return recall_score(labels, preds, average='macro')


In [11]:
results = []

reps = 3
for i in range(reps):

  model = ClassificationModel(
    "roberta", 
    "roberta-base",
    num_labels=7, 
    args={'num_train_epochs':2,
          'max_seq_length':256,
          'learning_rate':1e-5,
          'overwrite_output_dir': True, 
          'custom_callback': empty_cache
          }
  )

  model.train_model(train_df)

  result, model_outputs, wrong_predictions = model.eval_model(eval_df, 
                                                              acc=sklearn.metrics.accuracy_score,
                                                              f1=f1_multiclass,
                                                              precision=precision_multiclass,
                                                              recall=recall_multiclass,
                                                              )

  
  results.append({'run': i + 1, 'accuracy': result['acc'], 'f1': result['f1'], 'precision': result['precision'], 'recall': result['recall']})

print('accuracy, f1, precision, recall:')
for i in results:
  print(i['accuracy'], i['f1'], i['precision'], i['recall'])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/11160 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1395 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1395 [00:00<?, ?it/s]

...

Running Epoch 0 of 2:   0%|          | 0/1395 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1395 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/34 [00:00<?, ?it/s]

accuracy, f1, precision, recall:
0.6333333333333333 0.5470812342246671 0.5710902454837765 0.5500944098231095
0.6592592592592592 0.5522249755895483 0.5678353563647681 0.5508387657755317
0.6555555555555556 0.5577584056380152 0.5590995154656145 0.569836457503205
